<a href="https://colab.research.google.com/github/peiyaow/dl790/blob/master/hw1/train_dl_model_peiyao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

Using TensorFlow backend.


# Parameters

In [0]:
# using GPU
use_cuda = True
# model parameters
# N: batch size
N, D_in, H1, H2, D_out = 64, 28*28, 1024, 1024, 10
num_epochs = 20

# Define PrepareData class

In [0]:
# define prepare dataset
class PrepareData(Dataset):

    def __init__(self, X, y):
        if not torch.is_tensor(X):
            self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Download data

In [7]:
# loading data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train, x_test = np.array(x_train), np.array(x_test)
y_train, y_test = np.array(y_train), np.array(y_test)

# transform the data
n_train = x_train.shape[0]
n_test = x_test.shape[0]
x_train = x_train.reshape([n_train, 28*28])
x_test = x_test.reshape([n_test, 28*28])

ds = PrepareData(X=x_train, y=y_train)
ds = DataLoader(ds, batch_size=N, shuffle=True)

4423680/4422102 [==============================] - 0s 0us/step


# Define TwoLayerNet class

In [0]:
# define model
class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super(TwoLayerNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(D_in, H1),
            nn.Sigmoid(),
            nn.Linear(H1, H2),
            nn.Sigmoid(),
            nn.Linear(H2, D_out),
        )

    def forward(self, x):
        return self.model(x)
      
# dl model
model = TwoLayerNet(D_in, H1, H2, D_out)
model.cuda()

# Loss function and optimizer

In [0]:
# define loss function
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=lr)

# Train model

In [0]:
for t in range(num_epochs):
    for x, y in ds:
        x = Variable(x).float()
        y = Variable(y).long()
        
        if use_cuda and torch.cuda.is_available():
          x = x.cuda()
          y = y.cuda()
         
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)

        # Compute and print loss
        loss = loss_fn(y_pred, y)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Testing

In [0]:
ds_test = PrepareData(X=x_test, y=y_test)
ds_test = DataLoader(ds_test)

In [34]:
correct = 0
total = 0
for data in ds_test:
    x, y = data
    x, y = x.float().cuda(), y.long().cuda()
    outputs = model(x)
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 85 %
